# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
# importar a bibliotecas relevantes

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score

from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy



import warnings
warnings.filterwarnings('ignore')

In [2]:
#Carregando Dataframe
dataset = pd.read_csv('previsao_de_renda.csv')
dataset.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [510]:
dataset.nunique()

Unnamed: 0               15000
data_ref                    15
id_cliente                9845
sexo                         2
posse_de_veiculo             2
posse_de_imovel              2
qtd_filhos                   8
tipo_renda                   5
educacao                     5
estado_civil                 5
tipo_residencia              6
idade                       47
tempo_emprego             2589
qt_pessoas_residencia        9
renda                     9786
dtype: int64

In [511]:
dataset.columns

Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda'],
      dtype='object')

In [512]:
#Vamos remover as varáveis irrelevantes
cols_a_usar = ['sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda']
df_nova = dataset[cols_a_usar]
print(df_nova.shape)
df_nova.head()

(15000, 12)


,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [513]:
#Verificando NaN Values
df_nova.isna().sum()

sexo                        0
posse_de_veiculo            0
posse_de_imovel             0
qtd_filhos                  0
tipo_renda                  0
educacao                    0
estado_civil                0
tipo_residencia             0
idade                       0
tempo_emprego            2573
qt_pessoas_residencia       0
renda                       0
dtype: int64

In [514]:
# Substituir os NaN values na variável tempo_emprego por 0
df_nova['tempo_emprego'] = df_nova['tempo_emprego'].fillna(0)
df_nova.isna().sum()

sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
qt_pessoas_residencia    0
renda                    0
dtype: int64

In [515]:
# Verificando os datatypes
print(df_nova.dtypes)

sexo                      object
posse_de_veiculo            bool
posse_de_imovel             bool
qtd_filhos                 int64
tipo_renda                object
educacao                  object
estado_civil              object
tipo_residencia           object
idade                      int64
tempo_emprego            float64
qt_pessoas_residencia    float64
renda                    float64
dtype: object


In [516]:
#codificar as características categóricas
df_nova = pd.get_dummies(df_nova, drop_first=True)
print(df_nova.dtypes)

posse_de_veiculo                    bool
posse_de_imovel                     bool
qtd_filhos                         int64
idade                              int64
tempo_emprego                    float64
qt_pessoas_residencia            float64
renda                            float64
sexo_M                              bool
tipo_renda_Bolsista                 bool
tipo_renda_Empresário               bool
tipo_renda_Pensionista              bool
tipo_renda_Servidor público         bool
educacao_Pós graduação              bool
educacao_Secundário                 bool
educacao_Superior completo          bool
educacao_Superior incompleto        bool
estado_civil_Separado               bool
estado_civil_Solteiro               bool
estado_civil_União                  bool
estado_civil_Viúvo                  bool
tipo_residencia_Casa                bool
tipo_residencia_Com os pais         bool
tipo_residencia_Comunitário         bool
tipo_residencia_Estúdio             bool
tipo_residencia_

In [517]:
X = df_nova.drop('renda', axis=1)
y = df_nova['renda']
print(X.shape)
print(y.shape)


(15000, 24)
(15000,)


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

##### <span style="color:blue"> 1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento)

In [520]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.25, random_state=42)


##### <span style="color:blue"> 2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

In [522]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1, 100]


In [523]:
# Testar cada alpha com Lasso Regression (L1)
for alpha in alphas:
    model_1 = Ridge(alpha=alpha)
    model_1.fit(train_X, train_y)
    y_pred = model_1.predict(test_X)
    r2_Ridge = r2_score(test_y, y_pred)
    
    print(f"Alpha: {alpha:.4f} -> R²(ridge): {r2_Ridge:.7f}")

Alpha: 0.0000 -> R²(ridge): 0.2684577
Alpha: 0.0010 -> R²(ridge): 0.2684577
Alpha: 0.0050 -> R²(ridge): 0.2684578
Alpha: 0.0100 -> R²(ridge): 0.2684579
Alpha: 0.0500 -> R²(ridge): 0.2684585
Alpha: 0.1000 -> R²(ridge): 0.2684592
Alpha: 100.0000 -> R²(ridge): 0.2685357


##### <span style="color:green"> Observação: 
No output, todos os valores de ```𝑅2```  estão muito próximos, mas o maior deles é:

🔹 Alpha = 0.005 → 𝑅2=0.2684579


##### <span style="color:blue"> 3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?

In [526]:
# Testar cada alpha com Lasso Regression (L1)
for alpha in alphas:
    model_2 = Lasso(alpha=alpha)
    model_2.fit(train_X, train_y)
    y_pred = model_2.predict(test_X)
    r2_lasso = r2_score(test_y, y_pred)
    
    print(f"Alpha: {alpha:.4f} -> R²(lasso): {r2_lasso:.7f}")


Alpha: 0.0000 -> R²(lasso): 0.2684578
Alpha: 0.0010 -> R²(lasso): 0.2684579
Alpha: 0.0050 -> R²(lasso): 0.2684584
Alpha: 0.0100 -> R²(lasso): 0.2684590
Alpha: 0.0500 -> R²(lasso): 0.2684635
Alpha: 0.1000 -> R²(lasso): 0.2684684
Alpha: 100.0000 -> R²(lasso): 0.2639023


##### <span style="color:green"> Observação: 
O melhor resultado é o maior valor de ```𝑅2``` , pois indica a melhor explicação da variabilidade dos dados pelo modelo.

🔹 Alpha = 0.005 → 𝑅2=0.2684584


##### <span style="color:blue"> 4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?

In [529]:
df_nova.head()

,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,renda,sexo_M,tipo_renda_Bolsista,tipo_renda_Empresário,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,False,True,0,26,6.602740,1.0,8060.34,False,False,True,...,False,False,True,False,False,True,False,False,False,False
1,True,True,0,28,7.183562,2.0,1852.15,True,False,False,...,False,False,False,False,False,True,False,False,False,False
2,True,True,0,35,0.838356,2.0,2253.89,False,False,True,...,False,False,False,False,False,True,False,False,False,False
3,False,True,1,30,4.846575,3.0,6600.77,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,True,False,0,33,4.293151,1.0,6475.97,True,False,False,...,False,False,True,False,False,False,False,False,False,True


In [530]:
print(X.dtypes)

posse_de_veiculo                    bool
posse_de_imovel                     bool
qtd_filhos                         int64
idade                              int64
tempo_emprego                    float64
qt_pessoas_residencia            float64
sexo_M                              bool
tipo_renda_Bolsista                 bool
tipo_renda_Empresário               bool
tipo_renda_Pensionista              bool
tipo_renda_Servidor público         bool
educacao_Pós graduação              bool
educacao_Secundário                 bool
educacao_Superior completo          bool
educacao_Superior incompleto        bool
estado_civil_Separado               bool
estado_civil_Solteiro               bool
estado_civil_União                  bool
estado_civil_Viúvo                  bool
tipo_residencia_Casa                bool
tipo_residencia_Com os pais         bool
tipo_residencia_Comunitário         bool
tipo_residencia_Estúdio             bool
tipo_residencia_Governamental       bool
dtype: object


In [531]:
X = X.astype(float)  

In [532]:
####Executando o modelo proposto de stepwise
def stepwise_selection(X, y, 
                        initial_list=[], 
                        threshold_in=0.01, 
                        threshold_out = 0.05, 
                        verbose=True):
        """ Perform a forward-backward feature selection 
        based on p-value from statsmodels.api.OLS
        Arguments:
            X - pandas.DataFrame with candidate features
            y - list-like with the target
            initial_list - list of features to start with (column names of X)
            threshold_in - include a feature if its p-value < threshold_in
            threshold_out - exclude a feature if its p-value > threshold_out
            verbose - whether to print the sequence of inclusions and exclusions
        Returns: list of selected features 
        Always set threshold_in < threshold_out to avoid infinite looping.
        See https://en.wikipedia.org/wiki/Stepwise_regression for the details
        """
        included = list(initial_list)
        #print("included")
        #print(included)
       
        while True:
            changed=False
            # forward step
            excluded = list(set(X.columns)-set(included))
            new_pval = pd.Series(index=excluded)
            
            for new_column in excluded:
                model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
                print("modelo 1")
                print(model.summary())
                new_pval[new_column] = model.pvalues[new_column]
                print("new_pval")
                print(new_pval)
                
            best_pval = new_pval.min()
            print("best_pval")
            print(best_pval)      
            if best_pval < threshold_in:
                print("best_pval")
                best_feature = new_pval.idxmin()
                included.append(best_feature)
                changed=True
                if verbose:
                    print("Entrou")
                    print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))
              
            
            #sys.exit()
            # backward step
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
            print("modelo 2")
            print(model.summary())
            # use all coefs except intercept
            pvalues = model.pvalues.iloc[1:]
            worst_pval = pvalues.max() # null if pvalues is empty
            print("worst_pval")
            print(worst_pval) 
            if worst_pval > threshold_out:
                changed=True
                worst_feature = pvalues.idxmax()
                included.remove(worst_feature)
                if verbose:
                    print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
            if not changed:
                break
        return included

result = stepwise_selection(X, y)



modelo 1
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.081
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     1327.
Date:                Sun, 16 Mar 2025   Prob (F-statistic):          2.04e-278
Time:                        00:10:36   Log-Likelihood:            -1.5595e+05
No. Observations:               15000   AIC:                         3.119e+05
Df Residuals:                   14998   BIC:                         3.119e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4060.5833     78.773     51.548

In [533]:
reg_stepwise = sm.OLS(y, sm.add_constant(pd.DataFrame(X))).fit()
reg_stepwise.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     221.7
Date:                Sun, 16 Mar 2025   Prob (F-statistic):               0.00
Time:                        00:10:39   Log-Likelihood:            -1.5430e+05
No. Observations:               15000   AIC:                         3.087e+05
Df Residuals:                   14975   BIC:                         3.088e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                         -4878.6701   2229.392     -2.188      0.029   -9248.551    -508.789
posse_de_veiculo                  6.2182    129.843      0.048      0.962    -248.291     260.727
posse_de_imovel                 387.4028    127.962      3.027      0.002     136.581     638.224
qtd_filhos                    -1016.5059   1040.217     -0.977      0.328   -3055.458    1022.446
idade                            40.6791      7.559      5.381      0.000      25.862      55.497
tempo_emprego                   552.7427     10.172     54.338      0.000     532.804     572.681
qt_pessoas_residencia          1146.5441   1038.281      1.104      0.269    -888.613    3181.701
sexo_M                         5879.7270    137.106     42.885      0.000    5610.982    6148.472
tipo_renda_Bolsista           -1433.4495   2374.406     -0.604      0.546   -6087.576    3220.677
tipo_renda_Empresário           810.8514    146.884      5.520      0.000     522.940    1098.763
tipo_renda_Pensionista         2690.2177    236.476     11.376      0.000    2226.697    3153.739
tipo_renda_Servidor público     124.9884    218.593      0.572      0.567    -303.480     553.457
educacao_Pós graduação          625.2472   1504.206      0.416      0.678   -2323.181    3573.676
educacao_Secundário             154.1790    560.479      0.275      0.783    -944.429    1252.787
educacao_Superior completo      783.0261    564.938      1.386      0.166    -324.321    1890.373
educacao_Superior incompleto   -218.7200    631.907     -0.346      0.729   -1457.334    1019.894
estado_civil_Separado           982.7116   1067.829      0.920      0.357   -1110.363    3075.787
estado_civil_Solteiro           733.9742   1047.273      0.701      0.483   -1318.809    2786.757
estado_civil_União             -493.0130    228.952     -2.153      0.031    -941.787     -44.239
estado_civil_Viúvo              983.5765   1077.717      0.913      0.361   -1128.881    3096.034
tipo_residencia_Casa           -213.1983    517.574     -0.412      0.680   -1227.707     801.311
tipo_residencia_Com os pais    -288.5781    580.819     -0.497      0.619   -1427.055     849.899
tipo_residencia_Comunitário    -421.3319   1029.140     -0.409      0.682   -2438.573    1595.909
tipo_residencia_Estúdio         186.2067    935.434      0.199      0.842   -1647.359    2019.773
tipo_residencia_Governamental   360.9207    614.680      0.587      0.557    -843.928    1565.769
==============================================================================
Omnibus:                    21863.118   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         15345272.223
Skew:                           8.566   Prob(JB):                         0.00
Kurtosis:              

In [534]:
# Definir X e y
X = df_nova[result]
y = df_nova['renda']

X = np.asarray(X, dtype=np.float64)
y = np.asarray(y, dtype=np.float64)

In [535]:
# Dividir os dados em treino e teste
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.25, random_state=42)

# Adiciona a constante (intercepto)
train_X = sm.add_constant(train_X)
test_X = sm.add_constant(test_X)

# Ajusta o modelo OLS e faz previsões
model = sm.OLS(train_y, train_X).fit()
y_pred = model.predict(test_X)

# Calcula o R²
print(f'R-squared: {r2_score(test_y, y_pred):.4f}')

R-squared: 0.2690


In [536]:
# RIDGE FIT
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(train_y, train_X)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=0,  # ridge fit
                             alpha=alpha)
    y_pred = reg.predict(test_X)
    r2 = r2_score(test_y, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
pd.DataFrame({'alpha':alphas, 
              '𝑅2-Ridge':r2_y_pred
             }).sort_values(by='𝑅2-Ridge', ascending=False)

Alpha 0: 
R-squared = 0.26901118051565687

Alpha 0.001: 
R-squared = 0.2689469332209887

Alpha 0.005: 
R-squared = 0.26857355767672975

Alpha 0.01: 
R-squared = 0.26793239547468195

Alpha 0.05: 
R-squared = 0.2606171843325742

Alpha 0.1: 
R-squared = 0.2509899778318002



,alpha,𝑅2-Ridge
0,0.000,0.269011
1,0.001,0.268947
2,0.005,0.268574
3,0.010,0.267932
4,0.050,0.260617
5,0.100,0.250990


In [537]:
# LASSO FIT
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(train_y, train_X)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,  # lasso fit
                             alpha=alpha)
    y_pred = reg.predict(test_X)
    r2 = r2_score(test_y, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
pd.DataFrame({'alpha ':alphas, 
              '𝑅2-Lasso':r2_y_pred
             }).sort_values(by='𝑅2-Lasso', ascending=False)

Alpha 0: 
R-squared = 0.26901118051565687

Alpha 0.001: 
R-squared = 0.26901118051565687

Alpha 0.005: 
R-squared = 0.26901118051565687

Alpha 0.01: 
R-squared = 0.26901118051565687

Alpha 0.05: 
R-squared = 0.26901118051565687

Alpha 0.1: 
R-squared = 0.26901118051565687



,alpha,𝑅2-Lasso
0,0.000,0.269011
1,0.001,0.269011
2,0.005,0.269011
3,0.010,0.269011
4,0.050,0.269011
5,0.100,0.269011


##### <span style="color:blue"> 5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

##### <span style="color:green"> Observação: 
Os resultados da regressão Ridge variam com diferentes valores alfa, enquanto os resultados da regressão Lasso permanecem constantes para todos os valores de alfa.

- **Regressão Ridge**: A melhor pontuação R² é **0,2690** quando alfa = 0.
- **Regressão Lasso**: A pontuação R² permanece constante em **0,2690** para todos os valores alfa.

Como Lasso não melhora R² e Ridge atinge uma pontuação ligeiramente melhor em alfa = 0,001 (**0,2689**), **Ridge tem um desempenho ligeiramente melhor** neste caso, pois permite alguma flexibilidade no ajuste do modelo. No entanto, a melhoria é mínima.

##### <span style="color:blue"> 6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

variaveis_eficientes => ["sexo_M", "idade", "tempo_emprego", "tipo_renda_Pensionista", "tipo_renda_Empresário",
    "posse_de_imovel",
    "tipo_renda_Bolsista"
]

##### <span style="color:blue"> 7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.